In [ ]:
import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image

#from model.networks_tf import Generator
from model.networks import Generator

plt.rcParams['figure.facecolor'] = 'white'

## Load generator model


In [ ]:
use_cuda_if_available = True
device = torch.device('cuda' if torch.cuda.is_available() 
                             and use_cuda_if_available else 'cpu')

#sd_path = 'pretrained/states_tf_places2.pth'
sd_path = 'pretrained/states_pt_places2.pth'

generator = Generator(checkpoint=sd_path, return_flow=True).to(device)

## Load image and mask


In [ ]:
n = 1
image_path = f"examples/inpaint/case{n}.png"
mask_path = f"examples/inpaint/case{n}_mask.png"

image = Image.open(image_path)
mask = Image.open(mask_path)

## Plot raw image and mask


In [ ]:
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 10))
ax1.imshow(image)  # plot raw image
ax2.imshow(mask.__array__()[:,:,0])   # plot masked image
plt.show()


## Inpaint

In [ ]:
image_org = T.ToTensor()(image).to(device)
mask = T.ToTensor()(mask).to(device)

output = generator.infer(image_org, mask, return_vals=['inpainted', 'stage1', 'stage2', 'flow'])

## Results


In [ ]:
mask = (mask[0:1] > 0.).to(dtype=torch.float32)
image_masked = (image_org*(1-mask)).permute(1, 2, 0).cpu()

print("Result:")
plt.figure(figsize=(10, 8))
plt.imshow(output[0])
plt.show()

w, h = image.size

print("""| Raw | Masked |
| Stage1 | Stage2 |""")
_, axes = plt.subplots(2, 2, figsize=(15*w / max(w,h), 15*h / max(w,h)))
axes[0,0].imshow(image)
axes[0,1].imshow(image_masked)
axes[1,0].imshow(output[1])
axes[1,1].imshow(output[2])
plt.show()

### Plot attention flow map


In [ ]:
plt.imshow(output[3][0].cpu().permute(1, 2, 0))

## Test Contextual Attention


In [ ]:

from model.networks import ContextualAttention
#from model.networks_tf import ContextualAttention


contextual_attention = ContextualAttention(ksize=3, stride=1, rate=2, 
                                           fuse_k=3, softmax_scale=10, 
                                           fuse=False,
                                           return_flow=True)

imageB = 'examples/style_transfer/bike.jpg'
imageA = 'examples/style_transfer/bnw_butterfly.png'

def test_contextual_attention(imageA, imageB):
    """Test contextual attention layer with 3-channel image input
    (instead of n-channel feature).
    
    """
    rate = 2
    stride = 1
    grid = rate*stride
    
    b = Image.open(imageA)
    b = b.resize((b.width//2, b.height//2), resample=Image.BICUBIC)
    b = T.ToTensor()(b)

    _, h, w = b.shape
    b = b[:, :h//grid*grid, :w//grid*grid].unsqueeze(0)

    print(f"Size of imageA: {b.shape}")

    f = T.ToTensor()(Image.open(imageB)) 
    _, h, w = f.shape
    f = f[:, :h//grid*grid, :w//grid*grid].unsqueeze(0)
 
    print(f"Size of imageB: {f.shape}")   

    yt, flow = contextual_attention(f*255., b*255.)

    return yt, flow


yt, flow = test_contextual_attention(imageA, imageB)

_, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 10))
ax1.imshow(yt[0].permute(1, 2, 0)/255.)
ax2.imshow(flow[0].permute(1, 2, 0))
